In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


### Read the dataset

In [2]:
data_path = os.path.join('..', 'Data')
elect_csv =  os.path.join(data_path, 'electricity.csv')
df = pd.read_csv(elect_csv)
df.head(3)

,Continent,Year,Biofuel Electricity,Coal Electricity,Fossil Electricity,Gas Electricity,Hydro Electricity,Other Renewable Electricity,Energy per Capita,Annual CO2 Emissions per unit Energy (kg/kWh)
0,Asia,2000,27.54,2106.82,3693.59,931.16,564.26,44.55,9864.972,0.252652
1,Asia,2001,28.79,2248.81,3850.28,1005.01,609.76,45.28,10068.242,0.248706
2,Asia,2002,30.26,2454.56,4139.67,1067.54,634.35,46.97,10431.763,0.249808


In [3]:
idf = df.interactive()

In [4]:
year_slider = pn.widgets.IntSlider(name = 'Generation Year', start = 2000, end = 2022, step = 1, value = 2011)
year_slider

IntSlider(end=2022, name='Generation Year', start=2000, value=2011)

In [5]:
df.columns

Index(['Continent', 'Year', 'Biofuel Electricity', 'Coal Electricity',
       'Fossil Electricity', 'Gas Electricity', 'Hydro Electricity',
       'Other Renewable Electricity', 'Energy per Capita',
       'Annual CO2 Emissions per unit Energy (kg/kWh)'],
      dtype='object')

In [6]:
y_axis_electricity = pn.widgets.RadioButtonGroup(name = 'Y Axis', options = ['Hydro Electricity',
                     'Other Renewable Electricity'], button_type = 'success')
y_axis_electricity

RadioButtonGroup(button_type='success', name='Y Axis', options=['Hydro Electricity', ...], value='Hydro Electricity')

In [7]:
continents = ['Africa', 'Asia', 'Australia', 'Europe',
              'North America', 'South America']

### Combine widget with intereactive dataframe

In [8]:
electricity_pipeline = (
    idf[(idf.Year <= year_slider) &
    (idf.Continent.isin(continents))]
    .groupby(['Continent', 'Year'])[y_axis_electricity].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Year')
    .reset_index(drop =True)
)

electricity_pipeline

In [9]:
ect_plot = electricity_pipeline.hvplot.line(x = 'Year', by = 'Continent',
           y = y_axis_electricity, line_width = 2, cmap = 'Set3',
           title = "Electricity Generation by Continent")
ect_plot

### Energy per Capita

In [10]:
y_axis_kwh = pn.widgets.RadioButtonGroup(name = 'Y Axis', options = ['Energy per Capita'], 
                                         button_type = 'warning')
y_axis_kwh

RadioButtonGroup(button_type='warning', name='Y Axis', options=['Energy per Capita'], value='Energy per Capita')

In [11]:
kwh_pipeline = (
    idf[(idf.Year <= year_slider) &
    (idf.Continent.isin(continents))]
    .groupby(['Continent', 'Year'])[y_axis_kwh].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Year')
    .reset_index(drop =True)
)

kwh_pipeline

In [12]:
kwh_plot = kwh_pipeline.hvplot.line(x = 'Year', by = 'Continent',
           y = y_axis_kwh, line_width = 2, cmap = 'Set3',
           title = "Energy Generation per Capita")
kwh_plot

### Interactive Table

In [13]:
electricity_table = electricity_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote',
                    page_size = 10, sizing_mode = 'stretch_width')
electricity_table

### Create a multi-variable bar plot

Here, we compare the dataset based on the electricity generation type between the years 2000 to 2022 for the continents.

In [14]:
yaxis_electricity_source = pn.widgets.RadioButtonGroup(name = 'Y Axis', options = ['Biofuel Electricity', 'Coal Electricity',
       'Fossil Electricity', 'Gas Electricity'], button_type = 'primary')
yaxis_electricity_source

RadioButtonGroup(button_type='primary', name='Y Axis', options=['Biofuel Electricity', ...], value='Biofuel Electricity')

In [15]:
continents_world = ['Africa', 'Asia', 'Australia', 'Europe',
              'North America', 'South America']

electricity_bar_pipeline = (
    idf[(idf.Year <= year_slider) &
    (idf.Continent.isin(continents))]
    .groupby(['Continent', 'Year'])[yaxis_electricity_source].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = 'Year')
    .reset_index(drop =True)
)

electricity_bar_pipeline

Link the pipeline to dataset to generate the time-series comparative bar plot.

In [16]:
electricity_source_bar_plot = electricity_bar_pipeline.hvplot.bar(x = 'Continent',
           y = yaxis_electricity_source, line_width = 2, cmap = 'Set3',
           title = "Time-Series Electricity \nGenereation Continental Comparison")
electricity_source_bar_plot

# Final Dashboard

Combine all the plots into a single dashboard

In [17]:
template = pn.template.FastListTemplate(
    title = 'Electricity Generation by Continents',
    sidebar = [pn.pane.Markdown("# Electricity Generation and Emissions"),
               pn.pane.Markdown("#### Fossil fuels are one of the greatest contributions to pollution via Carbon IV oxide. The increased levels of Carbon IV Oxide result in higher world temperature, hence global warming. Global warming alters weather patterns leading to climate change. Since fossil fuels are used to generate electricity, it is vital to cut down their usage and switch to alternative electricity sources. Here, we present the amount of electricity generated by different continents from 2000 to 2022"),
               pn.pane.PNG('energy.png', sizing_mode = 'scale_both'),
               pn.pane.Markdown("## Settings"),
               year_slider],
    main = [pn.Row(pn.Column(y_axis_electricity, ect_plot.panel(width = 700), margin = (0, 25)),
                   electricity_table.panel(width = 500)),
            pn.Row(pn.Column(kwh_plot.panel(width = 700), margin = (0, 25)),
                   pn.Column(yaxis_electricity_source, electricity_source_bar_plot.panel(width = 600)))],
    accent_base_color = "#88d8b0",
    header_background = "#88d8b0",
)

template.servable();